In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression

import re
import string

import nltk
from nltk.stem import RSLPStemmer
nltk.download('rslp')
from nltk.corpus import stopwords
nltk.download('stopwords')
STOPWORDS = stopwords.words('portuguese')

from scipy.stats import loguniform
from pandas import read_csv
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

https://www.kaggle.com/code/mukeshmanral/predicting-customer-satisfaction

In [4]:
path='/kaggle/input/brazilian-ecommerce/'
customers_data = pd.read_csv(path + 'olist_customers_dataset.csv')
geolocation_data = pd.read_csv(path + 'olist_geolocation_dataset.csv')
order_items_data = pd.read_csv(path + 'olist_order_items_dataset.csv')
order_pay_data = pd.read_csv(path + 'olist_order_payments_dataset.csv')
order_revw_data = pd.read_csv(path + 'olist_order_reviews_dataset.csv')
orders_data = pd.read_csv(path + 'olist_orders_dataset.csv')
order_prd_data = pd.read_csv(path + 'olist_products_dataset.csv')
order_prd_cat_data = pd.read_csv(path + 'product_category_name_translation.csv')
order_sell_data = pd.read_csv(path + 'olist_sellers_dataset.csv')

In [5]:
all_datasets_name = [customers_data,geolocation_data,order_items_data,order_pay_data,order_revw_data,orders_data,order_prd_data,order_prd_cat_data,order_sell_data]
title_to_show = ['customers','geolocations','order_items','order_payments','order_revw','order','products','order_categories','order_sellers']
df = pd.DataFrame({})
df['df_name']=title_to_show
df['total_col_num']=[len(d.columns) for d in all_datasets_name]
df['total_row_num']=[len(d) for d in all_datasets_name]
df['col_name']=[(', ').join(d.columns) for d in all_datasets_name]
df.style.background_gradient()

In [6]:
#checking no of null values
df_null=pd.DataFrame({})
df_null['df_name']=title_to_show
#creating column of name of columns in dataset 
df_null['all_cols_num'] = [len(list(dataframe.columns))
                            for dataframe in all_datasets_name]
df_null['all_cols_name'] = [', '.join([col for col, null in dataframe.isnull().sum().items()])\
                            for dataframe in all_datasets_name]
df_null['null_sum']=[dataframe.isnull().sum().sum() for dataframe in all_datasets_name]
df_null['null_cols_name'] = [', '.join([col for col, null in dataframe.isnull().sum().items()
                                       if null >0]\
                                      )\
                            for dataframe in all_datasets_name]


df_null.style.background_gradient(cmap='Greys')
# df1['null_col_name']=[(', ').join(d.columns) for d in all_datasets_name]
# df1.style.background_gradient()
# customers_data.isnull().sum()

In [ ]:
# pandas profiling importing
!pip install Jinja2

In [ ]:
from jinja2 import escape
import pandas_profiling
order_revw_data.profile_report()

Sentiment Analysis using Logistic Regression
https://www.kaggle.com/code/marcosmota/sentiment-analysis-using-logistic-regression

In [7]:
order_review_sub=order_revw_data[['review_comment_message', 'review_score']]
order_review_sub.head()

In [8]:
#remove null
order_review_sub.dropna(inplace = True)
order_review_sub.isnull().sum()

In [10]:
#label df
order_review_sub['label']=pd.cut(x=order_review_sub['review_score'],\
                                 bins=[0,2,5], labels=[0,1])
order_review_sub.head()

In [11]:
order_review_sub = order_review_sub[['review_comment_message','label']]
reviews = order_review_sub.rename(columns={'review_comment_message':'text'})
reviews.head()

Preprocessing

In [12]:
#clean text

class ClearPipe(BaseEstimator, TransformerMixin):
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        texto_transformado = []
        for texto in X:
            texto = re.sub('\n', ' ', texto)
            
            texto = re.sub('\r', ' ', texto)

            texto = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', ' numero ', texto)

            texto = re.sub(r'R\$', ' ', texto)
            texto = re.sub(r'\W', ' ', texto)
            texto = re.sub(r'\s+', ' ', texto)

            urls = re.findall('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', texto)
            if len(urls) == 0:
                pass
            else:
                for url in urls:
                    for link in url:
                        texto = texto.replace(link, '')
                texto = texto.replace(':', '')
                texto = texto.replace('/', '')
          
            texto_transformado.append(texto)
        return texto_transformado

In [13]:
#stop words
class stop_words(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def _removeStopPipe(self, text):
        nopunc=[char for char in text if char not in string.punctuation]
        nopunc=''.join(nopunc)
        return [word for word in nopunc.split() if word.lower() not in STOPWORDS]
    def transform(self, X, y=None):
        #map(function, iterables):that works as an iterator to return a result after 
        #applying a function to every item of an iterable (tuple, lists, etc.).
        texto_transformado=list(map(lambda c: self._removeStopPipe(c), X))
        texto_transformado = list(map(lambda x: ' '.join(x), texto_transformado))
        return texto_transformado


In [14]:
#stemming
class stemming(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def _applyStemming(self, c):
        stemmer=RSLPStemmer()
        return list(map(lambda x:stemmer.stem(x), [word for word in c.split()]))
    def transform(self, X, y=None):
        #map(function, iterables):that works as an iterator to return a result after 
        #applying a function to every item of an iterable (tuple, lists, etc.).
        texto_transformado=list(map(lambda c: self._applyStemming(c), X))
        texto_transformado = list(map(lambda x: ' '.join(x), texto_transformado))
        return texto_transformado
        

In [15]:
pipe_preprocessamento = Pipeline([
    ('clear', ClearPipe()),
    ('stopwords', stop_words()),
    ('stemming', stemming()),
])
text_preprocessed = pipe_preprocessamento.fit_transform(reviews['text'])

In [36]:
#Vectorization
vectorizer = CountVectorizer(max_features=300, stop_words=STOPWORDS)
X = vectorizer.fit_transform(text_preprocessed).toarray()

In [37]:
from sklearn.model_selection import train_test_split

y=reviews['label'].values
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.3, random_state=42)
#rs=1 maintains the initial and last value and randomize the rest \
#while rs = 0 just does normal random of all data;the number 42 just corresponds to the seed of Randomness
#the supercomputer Deep Thought reveals that the answer to the great question of “life, the universe and everything” is 42.

In [47]:
print(reviews['text'].values)
print(X)
print(X_train)

In [ ]:
model = LogisticRegression( max_iter=10000,  dual=False)
model.fit(X_train, y_train,)

In [ ]:
acc = cross_val_score(model, X_test, y_test, cv=5, scoring='accuracy')
acc

In [ ]:
acc.mean()

In [ ]:
import seaborn as sns
sns.set(style='whitegrid')
sns.boxplot(data=acc,orient='v', width=0.35).set(xlabel ="Logistic Regression", title ='Model Performance (Accuracy)')

# fig = plt.figure()
# fig.suptitle('Model Performance (Accuracy)')
# ax = fig.add_subplot(111)
# ax.set_xticklabels(['Logistic Regression'])
# plt.boxplot(acc)
# plt.show()

In [ ]:
def predict(text):
    text_preprocessed = pipe_preprocessamento.fit_transform(text)
    text_transformed = vectorizer.transform(text_preprocessed).toarray()
    pred = model.predict(text_transformed)
    return 'positive' if pred[0] == 1 else 'negative'

In [ ]:
text1 = ['produto de péssima qualidade.']
print(predict(text1))
text2 = ['Excelente produto, indico']
print(predict(text2))

# Hyperparameters Optimization

# What Steps To Follow For Hyper Parameter Tuning?
* Select the type of model we want to use like RandomForestClassifier, regressor or any other model
* Check what are the parameters of the model
* Select the methods for searching the hyperparameter
* Select the cross-validation approach
* Evaluate the model using the score

https://analyticsindiamag.com/guide-to-hyperparameters-tuning-using-gridsearchcv-and-randomizedsearchcv/

In [ ]:
#Repeated Stratified K-Fold cross validator.
#Repeats Stratified K-Fold n times with different randomization in each repetition.
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

#For small datasets,
#‘liblinear’ is a good choice, whereas ‘sag’ and ‘saga’ are faster for large ones;
#For multiclass problems, 
#only ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ handle multinomial loss;
#‘liblinear’ is limited to one-versus-rest schemes.

#Cfloat, default=1.0
#Inverse of regularization strength; must be a positive float. 
#Like in support vector machines, smaller values specify stronger regularization.

space = dict()
space['solver'] = ['lbfgs', 'liblinear']
space['penalty'] = ['none', 'l1', 'l2']
space['C'] = loguniform(1e-5, 100)

The only difference between both the approaches is in grid search we define the combinations and do training of the model whereas in RandomizedSearchCV the model selects the combinations randomly. Both are very effective ways of tuning the parameters that increase the model generalizability.


# RandomizedSearchCV

In [ ]:
#class sklearn.model_selection.RandomizedSearchCV \
# (estimator, param_distributions, *, n_iter=10, scoring=None, n_jobs=None, \
#refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', random_state=None, \
#error_score=nan, return_train_score=False)

cv = RepeatedStratifiedKFold(n_splits=10, random_state=42)
search = RandomizedSearchCV(model, space, scoring='accuracy', n_jobs=4, \
                            cv=cv, random_state=42)

In [ ]:
# execute search
result = search.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

# GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
g_space = dict()
# g_space['solver'] = ['lbfgs', 'liblinear']
g_space['penalty'] = ['none', 'l1', 'l2']
# g_space['C'] = loguniform(1e-5, 100)
g_space['C'] = [1, 10]
# print(g_space)
# parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
g_cv = RepeatedStratifiedKFold(n_splits=10)

g_search = GridSearchCV(model, g_space, scoring='accuracy', n_jobs=4,\
                       cv=g_cv)
# execute search
g_result = g_search.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % g_result.best_score_)
print('Best Hyperparameters: %s' % g_result.best_params_)

RandomizedSearchCV

Best Score: 0.8723179476372497

Best Hyperparameters: {'C': 0.004185822729546973, 'penalty': 'none', 'solver': 'lbfgs'}

GridSearchCV

Best Score: 0.8722379589834681

Best Hyperparameters: {'C': 1, 'penalty': 'none'}



# multiple classifiers

https://www.kaggle.com/code/haijie/prediction-of-the-customers-reviews

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, cross_val_score

clf_lr = LogisticRegression(random_state=22, max_iter=2000)
clf_rf = RandomForestClassifier(random_state=22)
clf_svc = SVC(random_state=22)

In [ ]:
param_grid_lr = {'solver': ['liblinear', 'lbfgs'], 'C': np.logspace(-4, 2, 9)}

param_grid_rf = [{'n_estimators': [10, 50], 
                  'min_samples_leaf': [1, 3, 5], 
                  'max_features': ['sqrt', 'log2']}]

param_grid_svc = [{'kernel': ['rbf'], 'C': np.logspace(-4, 4, 9), 'gamma': np.logspace(-4, 0, 4)},
                  {'kernel': ['linear'], 'C': np.logspace(-4, 4, 9)}]
param_grid_svc

In [ ]:
gridcvs = {}

for pgrid, clf, name in zip((param_grid_lr, param_grid_rf, param_grid_svc),
                            (clf_lr, clf_rf, clf_svc),
                            ('lr', 'RF', 'SVM')):
    gcv = GridSearchCV(clf, pgrid, cv=3, refit=True)
    gridcvs[name] = gcv
gridcvs

In [ ]:
gridcvs_RF = {}
gcv = GridSearchCV(clf_rf, param_grid_rf, cv=3, refit=True)
gridcvs_RF['RF'] = gcv
# for pgrid, clf, name in zip(param_grid_rf,clf_rf,'RF'):
#     gcv = GridSearchCV(clf, pgrid, cv=3, refit=True)
#     gridcvs[name] = gcv
gridcvs_RF

In [ ]:
outer_cv = StratifiedKFold(n_splits=3, shuffle=True)
outer_scores = {}
# Evaluate a score by cross-validation.
for name, gs in gridcvs_RF.items():
    nested_score = cross_val_score(gs, X_train, y_train, cv=outer_cv)
    outer_scores[name] = nested_score
    print(f'{name}: outer accuracy {100*nested_score.mean():.2f} +/- {100*nested_score.std():.2f}')

In [ ]:
from sklearn.metrics import accuracy_score

final_clf = gridcvs_RF['RF']
final_clf.fit(X_train, y_train)

print(f'Best Parameters: {final_clf.best_params_}')

train_acc = accuracy_score(y_true=y_train, y_pred=final_clf.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=final_clf.predict(X_test))

print(f'Training Accuracy: {100*train_acc:.2f}')
print(f'Test Accuracy: {100*test_acc:.2f}')

RF: outer accuracy 87.19 +/- 0.32

Best Parameters: {'max_features': 'log2', 'min_samples_leaf': 1, 'n_estimators': 50}

Training Accuracy: 98.17

Test Accuracy: 87.67

# Sentiment Analysis using LSTM

https://www.kaggle.com/code/marcosmota/sentiment-analysis-using-lstm

https://www.kaggle.com/code/odins0n/feedback-based-rating-predictions

In [48]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

from argparse import Namespace


In [49]:
args = Namespace(
    dataset_csv = '/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv',
    train_split = 0.8,
    random_state = 42,
    vocab_size = 10000,
    embedding_dim = 16,
    max_length = 300,
    batch_size=128,
    num_epochs=5,
    early_stopping_criteria=5,
    dropout_p=0.3,
    model_storage="model_storage/lstm",
)

In [60]:
from sklearn.model_selection import train_test_split

y_1=reviews['label'].values
X_1=reviews['text'].values
X_train_1, X_test_1, y_train_1, y_test_1=train_test_split(X_1,y_1, test_size=0.3, random_state=42)
X_train_1

In [61]:
oov_tok = "<OOV>"
# by turning each text into either a sequence of integers
#oov_token: if given, it will be added to word_index and used to         
#replace out-of-vocabulary words during text_to_sequence calls
tokenizer = Tokenizer(num_words = args.vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train_1)

def preprocess(X_train, X_test, max_length, vocab_size, trunc_type='post', oov_tok = "<OOV>"):
    """
    Args
        training_sentences
        training_labels
        testing_sentences
        testing_labels
    Return
        training_sentences
        training_labels
        testing_sentences
        testing_labels 
    """
    training_sentences = tokenizer.texts_to_sequences(X_train)
#     print("sequences : ",training_sentences,'\n')
#     print("word_index : ",tokenizer.word_index)
    training_padded = pad_sequences(training_sentences,maxlen=max_length, truncating=trunc_type)
    
    testing_sequences = tokenizer.texts_to_sequences(X_test)
    testing_padded = pad_sequences(testing_sequences,maxlen=max_length)
    return training_padded, testing_padded

In [62]:
X_train_1, X_test_1 = preprocess(\
                X_train_1, X_test_1, args.max_length, args.vocab_size)

In [54]:
print(X_train_1)

In [63]:
def create_model():
    input = tf.keras.Input(shape=(args.max_length))
    x = tf.keras.layers.Embedding(input_dim=args.vocab_size, \
                                  output_dim=args.embedding_dim, \
                                  input_length=args.max_length)(input)

    x = tf.keras.layers.LSTM(16, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(16)(x)

    x = tf.keras.layers.Dropout(0.6)(x)

    x = tf.keras.layers.Dense(32, activation='relu')(x)

    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    return tf.keras.Model(input, output)

In [64]:
earlyStoppingCallback = tf.keras.callbacks.EarlyStopping(monitor='loss', \
                                                         patience=args.early_stopping_criteria)

model = create_model()
model.compile(
  loss = tf.keras.losses.BinaryCrossentropy(),
  optimizer= tf.keras.optimizers.Adam(
    learning_rate=0.0001),
  metrics=['accuracy']
)

history = model.fit(
    X_train_1,
    np.array(y_train_1), 
    validation_data=(X_test_1, np.array(y_test_1)),
    epochs=30,
    batch_size=args.batch_size,
    callbacks= [earlyStoppingCallback],
)

In [65]:
plt.figure(figsize = (10,7))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy vs Number of Epochs', fontsize = 16)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [66]:
plt.figure(figsize = (10,7))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss vs Number of Epochs', fontsize = 16)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Evaluation Model

In [67]:
# y_pred = model.predict(y_train)


y_predicted_prob = model.predict(X_test_1, verbose=0)
y_predicted_classes = np.where(y_predicted_prob > 0.5, 1, 0)

y_predicted_prob = y_predicted_prob[:, 0]
y_predicted_classes = y_predicted_classes[:, 0]

In [68]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

accuracy = accuracy_score(y_test_1, y_predicted_classes)
print('Accuracy: %f' % accuracy)

precision = precision_score(y_test_1, y_predicted_classes)
print('Precision: %f' % precision)

recall = recall_score(y_test_1, y_predicted_classes)
print('Recall: %f' % recall)

f1 = f1_score(y_test_1, y_predicted_classes)
print('F1 score: %f' % f1)

In [69]:
import seaborn as sns
# 0: <3; 1:>=3
data = confusion_matrix(y_test_1, y_predicted_classes)
df_cm = pd.DataFrame(data, columns=np.unique(y_predicted_classes), index = np.unique(y_predicted_classes))
df_cm.index.name = 'Ground Truth'
df_cm.columns.name = 'Prediction'
plt.figure(figsize = (10,7))
plt.title('Confusion Metrics', fontsize = 20)

sns.set(font_scale=1.4)
ax = sns.heatmap(df_cm, cbar=False, cmap="Blues", annot=True, annot_kws={"size": 16}, fmt='g')